In [1]:
!pip install -qqq pyautogen~=0.1.0 flaml[automl] openai langchain chromadb sentence-transformers requests bs4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.2/295.2 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 448.1/448.1 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 41.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
%%writefile airbnb_search.py
import requests
from bs4 import BeautifulSoup

# Step 1: Define the URL
url = "https://www.airbnb.com/"

# Step 2: Define search parameters
search_params = {
    "location": "Austin, Texas",
    "checkin": "2023-10-10",
    "checkout": "2023-10-11"
}

# Step 3: Send a GET request to Airbnb with the search parameters
response = requests.get(url, params=search_params)
response.raise_for_status()

# Step 4: Parse the HTML content of the page using BeautifulSoup
soup = BeautifulSoup(response.content, "html.parser")

# Step 5: Find and extract the Airbnb listings
results = soup.find_all("div", class_="_gig1e7")

# Step 6: Print the titles and prices of the listings
for result in results:
    title = result.find("div", class_="_1c2n35az")
    price = result.find("span", class_="_olc9rf0")

    if title and price:
        print("Title:", title.get_text())
        print("Price:", price.get_text())
        print()

Writing airbnb_search.py


In [2]:
import json

# Create a list of OpenAI configuration settings
config_list = [
  {
    "model": "gpt-3.5-turbo",
    "api_key": "",
  }
]

# Save the configuration list to a file
with open("OAI_CONFIG_LIST.json", "w") as f:
    json.dump(config_list, f)

In [6]:
import autogen

config_list = autogen.config_list_from_json(
    env_or_file="OAI_CONFIG_LIST.json",
    file_location=".",
)

assert len(config_list) > 0
print("models to use: ", [config_list[i]["model"] for i in range(len(config_list))])

llm_config={
    "request_timeout": 600,
    "seed": 44,                     # for caching and reproducibility
    "config_list": config_list,     # which models to use
    "temperature": 0,               # for sampling
}

agent_assistant = autogen.AssistantAgent(
    name="agent_assistant",
    llm_config=llm_config,
)

agent_proxy = autogen.UserProxyAgent(
    name="agent_proxy",
    human_input_mode="NEVER",           # NEVER, TERMINATE, or ALWAYS
                                            # TERMINATE - human input needed when assistant sends TERMINATE
    max_consecutive_auto_reply=10,
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={
        "work_dir": "agent_output",     # path for file output of program
        "use_docker": False,            # True or image name like "python:3" to use docker image
    },
    llm_config=llm_config,
    system_message="""Reply TERMINATE if the task has been solved at full satisfaction.
                      Otherwise, reply CONTINUE, or the reason why the task is not solved yet."""
)

agent_proxy.initiate_chat(
    agent_assistant,
    message="""I need you to write a python script that will do the following:
    1. go to airbnb
    2. search for an Buffalo New York stay from Oct 10, 2023 - Oct 11, 2023
    3. gather the results, no more than 10. The class html div to search for is "c4mnd7m dir dir-ltr".
    4. print that result to the screen
    """,
)

agent_proxy.send(
    recipient=agent_assistant,
    message="""Please save the python script you created to air.py"""
)

models to use:  ['gpt-3.5-turbo']
agent_proxy (to agent_assistant):

I need you to write a python script that will do the following:
    1. go to airbnb
    2. search for an Buffalo New York stay from Oct 10, 2023 - Oct 11, 2023
    3. gather the results, no more than 10. The class html div to search for is "c4mnd7m dir dir-ltr".
    4. print that result to the screen
    

--------------------------------------------------------------------------------
agent_assistant (to agent_proxy):

# filename: airbnb_search.py

import requests
from bs4 import BeautifulSoup

# Step 1: Go to Airbnb
url = "https://www.airbnb.com/"

# Step 2: Search for Buffalo New York stay from Oct 10, 2023 - Oct 11, 2023
search_params = {
    "location": "Buffalo, New York",
    "checkin": "2023-10-10",
    "checkout": "2023-10-11"
}

response = requests.get(url, params=search_params)
response.raise_for_status()

# Step 3: Gather the results
soup = BeautifulSoup(response.content, "html.parser")
results = soup.find

In [4]:
agent_proxy.initiate_chat(
    agent_assistant,
    message="""
Who should read this paper: https://arxiv.org/abs/2308.08155
""",
)

agent_proxy (to agent_assistant):


Who should read this paper: https://arxiv.org/abs/2308.08155


--------------------------------------------------------------------------------
agent_assistant (to agent_proxy):

To determine who should read a paper, we can look at the title, abstract, and keywords of the paper. However, as an AI assistant, I am unable to browse the web directly. Therefore, I suggest you open the link and provide me with the relevant information from the paper, such as the title, abstract, and keywords. Once you provide me with that information, I can help you determine who should read the paper.

--------------------------------------------------------------------------------
agent_proxy (to agent_assistant):

I apologize for the confusion, but as an AI language model, my responses are generated based on pre-existing knowledge and I don't have direct access to browse the internet or open specific links. Therefore, I am unable to provide you with the information from